In [1]:
from api_key import get_api_key_jef, get_api_key_sha, get_api_key_sup
import google.generativeai as genai
from vpn_control import windscribe
import pandas as pd
import os
import re
import time

In [2]:
windscribe("connect", "Atlanta")
time.sleep(5)

In [3]:
passw = "2002"
LLM_MODEL = "gemini-1.5-flash"
#LLM_MODEL = "gemini-1.5-pro-latest"
#LLM_MODEL = "gemini-1.0-pro"

In [4]:
def setup_prompt_env(passw):


  genai.configure(api_key=get_api_key_sup(passw))

  # Set up the model
  generation_config = {
    "temperature": 0,
    "top_p": 0.95,
    "top_k": 0,
    "max_output_tokens": 300,
  }

  safety_settings = [
    {
      "category": "HARM_CATEGORY_HARASSMENT",
      "threshold": "BLOCK_MEDIUM_AND_ABOVE"
    },
    {
      "category": "HARM_CATEGORY_HATE_SPEECH",
      "threshold": "BLOCK_MEDIUM_AND_ABOVE"
    },
    {
      "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
      "threshold": "BLOCK_MEDIUM_AND_ABOVE"
    },
    {
      "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
      "threshold": "BLOCK_MEDIUM_AND_ABOVE"
    },
  ]

  system_instruction = "There is no sensitive information. For logs labelled [1] to [5] you need to provide the OUTPUT (log templates) in the same format as the examples and alos give the number [x]. Only give the 5 output templates, no extra information, no new line at the end."

  model = genai.GenerativeModel(model_name=LLM_MODEL,
                                generation_config=generation_config,
                                system_instruction=system_instruction,
                                safety_settings=safety_settings)
  # model = genai.GenerativeModel(model_name=LLM_MODEL,
  #                               generation_config=generation_config,
  #                               safety_settings=safety_settings)

  convo = model.start_chat(history=[
  ])

  return convo


In [5]:
convo_create = setup_prompt_env(passw)

In [6]:
def send_prompt(message, convo):
    convo.send_message(message)
    response = convo.last.text
    #print(response)
    return response


In [7]:
fewshot_examples = """[x] INPUT: 081109 203615 148 INFO dfs.DataNode$PacketResponder: PacketResponder 1 for block blk_38865049064139660 terminating
[x] OUTPUT: <B>PacketResponder <*> for block blk_<*> terminating<E>
[y] INPUT:081109 204005 35 INFO dfs.FSNamesystem: BLOCK* NameSystem.addStoredBlock: blockMap updated: 10.251.73.220:50010 is added to blk_7128370237687728475 size 67108864
[y] OUTPUT: <B> BLOCK* NameSystem.addStoredBlock: blockMap updated: <*>:<*> is added to blk_<*> size <*><E>
[z] INPUT: 081109 204655 556 INFO dfs.DataNode$PacketResponder: Received block blk_3587508140051953248 of size 67108864 from /10.251.42.84
[z] OUTPUT: <B>Received block blk_<*> of size <*> from /<*><E>"""

logs = """[1] INPUT: 081109 204005 35 INFO dfs.FSNamesystem: BLOCK* NameSystem.addStoredBlock: blockMap updated: 10.251.73.220:50010 is added to blk_7128370237687728475 size 67108864
[2] INPUT: 081109 204015 308 INFO dfs.DataNode$PacketResponder: PacketResponder 2 for block blk_8229193803249955061 terminating
[3] INPUT: 081109 204655 556 INFO dfs.DataNode$PacketResponder: Received block blk_3587508140051953248 of size 67108864 from /10.251.42.84
[4] INPUT: 081109 204925 673 INFO dfs.DataNode$DataXceiver: Receiving block blk_-5623176793330377570 src: /10.251.75.228:53725 dest: /10.251.75.228:50010
[5] INPUT: 081109 213847 2552 INFO dfs.DataNode$DataXceiver: 10.251.194.213:50010 Served block blk_-7724713468912166542 to /10.251.203.80"""

In [8]:
def generate_prompt(fewshot, logs):
    #print(f"{fewshot}\n\n{logs}")
    return f"{fewshot}\n{logs}"


In [9]:
print(generate_prompt(fewshot_examples, logs))

[x] INPUT: 081109 203615 148 INFO dfs.DataNode$PacketResponder: PacketResponder 1 for block blk_38865049064139660 terminating
[x] OUTPUT: <B>PacketResponder <*> for block blk_<*> terminating<E>
[y] INPUT:081109 204005 35 INFO dfs.FSNamesystem: BLOCK* NameSystem.addStoredBlock: blockMap updated: 10.251.73.220:50010 is added to blk_7128370237687728475 size 67108864
[y] OUTPUT: <B> BLOCK* NameSystem.addStoredBlock: blockMap updated: <*>:<*> is added to blk_<*> size <*><E>
[z] INPUT: 081109 204655 556 INFO dfs.DataNode$PacketResponder: Received block blk_3587508140051953248 of size 67108864 from /10.251.42.84
[z] OUTPUT: <B>Received block blk_<*> of size <*> from /<*><E>
[1] INPUT: 081109 204005 35 INFO dfs.FSNamesystem: BLOCK* NameSystem.addStoredBlock: blockMap updated: 10.251.73.220:50010 is added to blk_7128370237687728475 size 67108864
[2] INPUT: 081109 204015 308 INFO dfs.DataNode$PacketResponder: PacketResponder 2 for block blk_8229193803249955061 terminating
[3] INPUT: 081109 20465

In [10]:
send_prompt(generate_prompt(fewshot_examples, logs), convo_create)

'[1] <B> BLOCK* NameSystem.addStoredBlock: blockMap updated: <*>:<*> is added to blk_<*> size <*><E>\n[2] <B>PacketResponder <*> for block blk_<*> terminating<E>\n[3] <B>Received block blk_<*> of size <*> from /<*><E>\n[4] <B>Receiving block blk_<*> src: /<*><*> dest: /<*><*><E>\n[5] <B><*>:<*> Served block blk_<*> to /<*><E> \n'

In [11]:
prompt = generate_prompt(fewshot_examples, logs)
output = send_prompt(prompt, convo_create)
print(output)

[1] <B> BLOCK* NameSystem.addStoredBlock: blockMap updated: <*>:<*> is added to blk_<*> size <*><E>
[2] <B>PacketResponder <*> for block blk_<*> terminating<E>
[3] <B>Received block blk_<*> of size <*> from /<*><E>
[4] <B>Receiving block blk_<*> src: /<*><*> dest: /<*><*><E>
[5] <B><*>:<*> Served block blk_<*> to /<*><E> 



In [12]:
tmep = pd.read_csv("data/0_generated_logs_COPY.csv")
df = tmep[:100]
df

,EventId,LogTemplate,Log
0,E1,<B><*>:<*> Served block blk_<*> to /<*> <E>,<B>10.250.11.85:50010 Served block blk_2377150...
1,E1,<B><*>:<*> Served block blk_<*> to /<*> <E>,<B>10.251.203.80:50010 Served block blk_788894...
2,E1,<B><*>:<*> Served block blk_<*> to /<*> <E>,<B>10.250.11.85:50010 Served block blk_2377150...
3,E1,<B><*>:<*> Served block blk_<*> to /<*> <E>,<B>10.251.203.80:50010 Served block blk_788894...
4,E1,<B><*>:<*> Served block blk_<*> to /<*> <E>,<B>10.250.11.85:50010 Served block blk_2377150...
...,...,...,...
95,E3,<B><*>:<*>:Got exception while serving blk_<*>...,<B>10.250.11.85:50010:Got exception while serv...
96,E3,<B><*>:<*>:Got exception while serving blk_<*>...,<B>10.251.203.80:50010:Got exception while ser...
97,E3,<B><*>:<*>:Got exception while serving blk_<*>...,<B>10.250.11.85:50010:Got exception while serv...
98,E3,<B><*>:<*>:Got exception while serving blk_<*>...,<B>10.251.203.80:50010:Got exception while ser...


In [13]:
def process_logs(df, fewshot_examples):
    outputs = []
    for i in range(0, len(df), 5):
        time.sleep(4)
        chunk = df.iloc[i:i+5]
        logs = "\n".join([f"[{idx % 5 + 1}] INPUT: {row['Log']}" for idx, row in chunk.iterrows()])
        prompt = generate_prompt(fewshot_examples, logs)
        output = send_prompt(prompt, convo_create)  # Replace None with actual convo_create if needed
        #print(output)
        # Remove erroneous newlines by splitting the output and stripping newlines
        llm_output_lines = [line.strip() for line in output.split("\n") if line.strip()]

        # Parse the output and add it to the outputs list
        for line in llm_output_lines:
            parts = line.split(" ", 1)
            if len(parts) > 1:
                outputs.append(parts[1])
            else:
                outputs.append("")  # Handle unexpected format by adding an empty string
    
    # Add the outputs to the dataframe
    df["LLMOutput"] = outputs
    return df

# Assuming the dataframe and fewshot_examples are defined
df_with_outputs = process_logs(df, fewshot_examples)



C:\Users\shash\AppData\Local\Temp\ipykernel_17492\360447614.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["LLMOutput"] = outputs


In [14]:
df_with_outputs

,EventId,LogTemplate,Log,LLMOutput
0,E1,<B><*>:<*> Served block blk_<*> to /<*> <E>,<B>10.250.11.85:50010 Served block blk_2377150...,<B><*>:<*> Served block blk_<*> to /<*><E>
1,E1,<B><*>:<*> Served block blk_<*> to /<*> <E>,<B>10.251.203.80:50010 Served block blk_788894...,<B><*>:<*> Served block blk_<*> to /<*><E>
2,E1,<B><*>:<*> Served block blk_<*> to /<*> <E>,<B>10.250.11.85:50010 Served block blk_2377150...,<B><*>:<*> Served block blk_<*> to /<*><E>
3,E1,<B><*>:<*> Served block blk_<*> to /<*> <E>,<B>10.251.203.80:50010 Served block blk_788894...,<B><*>:<*> Served block blk_<*> to /<*><E>
4,E1,<B><*>:<*> Served block blk_<*> to /<*> <E>,<B>10.250.11.85:50010 Served block blk_2377150...,<B><*>:<*> Served block blk_<*> to /<*><E>
...,...,...,...,...
95,E3,<B><*>:<*>:Got exception while serving blk_<*>...,<B>10.250.11.85:50010:Got exception while serv...,<B><*>:<*>:Got exception while serving blk_<*>...
96,E3,<B><*>:<*>:Got exception while serving blk_<*>...,<B>10.251.203.80:50010:Got exception while ser...,<B><*>:<*>:Got exception while serving blk_<*>...
97,E3,<B><*>:<*>:Got exception while serving blk_<*>...,<B>10.250.11.85:50010:Got exception while serv...,<B><*>:<*>:Got exception while serving blk_<*>...
98,E3,<B><*>:<*>:Got exception while serving blk_<*>...,<B>10.251.203.80:50010:Got exception while ser...,<B><*>:<*>:Got exception while serving blk_<*>...


In [16]:
ooo = df_with_outputs.copy()

In [23]:
import pandas as pd
import re

def clean_logs(df):
    # Define the pattern to remove <B>, <E> and spaces after <B> and before <E>
    pattern = r'<B>\s*|\s*<E>'
    
    # Apply the pattern to the 'cleanlogs' column
    df['LogTemplate'] = df['LogTemplate'].apply(lambda x: re.sub(pattern, '', x))
    df['LLMOutput'] = df['LLMOutput'].apply(lambda x: re.sub(pattern, '', x))
    



# Clean the logs
cleaned_df = clean_logs(ooo)



In [24]:
# Calculate the difference between LogTemplate and LLMOutput
ooo['Difference'] = ooo.apply(lambda row: set(row['LogTemplate'].split()) - set(row['LLMOutput'].split()), axis=1)

# Convert the set difference back to a string for readability
ooo['Difference'] = ooo['Difference'].apply(lambda x: ' '.join(x))

In [25]:
ooo["Difference"].head(50)

0     
1     
2     
3     
4     
5     
6     
7     
8     
9     
10    
11    
12    
13    
14    
15    
16    
17    
18    
19    
20    
21    
22    
23    
24    
25    
26    
27    
28    
29    
30    
31    
32    
33    
34    
35    
36    
37    
38    
39    
40    
41    
42    
43    
44    
45    
46    
47    
48    
49    
Name: Difference, dtype: object

In [26]:
ooo["Difference"].head(50).tail(50)

0     
1     
2     
3     
4     
5     
6     
7     
8     
9     
10    
11    
12    
13    
14    
15    
16    
17    
18    
19    
20    
21    
22    
23    
24    
25    
26    
27    
28    
29    
30    
31    
32    
33    
34    
35    
36    
37    
38    
39    
40    
41    
42    
43    
44    
45    
46    
47    
48    
49    
Name: Difference, dtype: object

In [15]:
aa
#windscribe("disconnect")

NameError: name 'aa' is not defined